In [7]:
# ------------------------------------------------ #
# IMPORT LIBRARIES AND SET VARIABLES
# ------------------------------------------------ #

# Adapted from https://dev.socrata.com/foundry/data.ny.gov/wujg-7c2s 

# --- imports ---- #

import pandas as pd
from sodapy import Socrata

# handle secrets
from dotenv import load_dotenv
import os
load_dotenv()

# nyc data.ny.gov app token
my_token = os.getenv("API_TOKEN")


# ------------------------------------------------ #
# CHANGE THESE VARIABLES FOR STATION YOU WANT
# ------------------------------------------------ #
# Need to make folders for station #s manually first in these folders:
# ./data/cleaned/
# ./data/raw/

# Filesnames to save to
station_id = '636' # this is used in the search string 
station_name = '636_JayStMetrotech_2023' # this is just for saving a file name



In [8]:
# ------------------------------------------------ #
# MAKE CALL TO API AND TURN INTO DATAFRAME
# ------------------------------------------------ #

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.ny.gov", my_token)


months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
enddates = ['31', '28', '31', '30', '31', '30', '31', '31', '30', '31', '30', '31'] #last day of the month in 2023


for i in range(len(months)):
    m = months[i]
    dlast = enddates[i]

    save_string_raw = './data/2023/raw/' +station_id+ '/'+ station_name +'_' + m + '_raw.csv'
    save_string_cleaned = './data/2023/cleaned/' +station_id+ '/' +station_name +'_' + m + '_cleaned.csv'
    
    print(m + dlast)

    results = client.get("wujg-7c2s", 
                     limit=50000,
                     where="station_complex_id = '" +station_id+ "' AND transit_timestamp between '2023-" +m+ "-01T00:00:00' and '2023-" +m+ "-" + dlast + "T23:00:00'")

    # Convert to pandas DataFrame
    results_df = pd.DataFrame.from_records(results)

    # ------------------------------------------------ #
    # WRITE RAW TO CSV
    # ------------------------------------------------ #
    # need to manually make folder for station id first!
    
    results_df.to_csv(save_string_raw, index = False)

    # ------------------------------------------------ #
    # REMOVE NA & UNNECESSARY COLUMNS
    # ------------------------------------------------ #
    
    # remove rows with any NaNs -- seems to me that the data is relatively clean
    results_clean = results_df.dropna()
    
    results_clean = results_clean.drop(columns=['latitude',
                                         'longitude',
                                         'georeference',
                                         'transit_mode',
                                         'payment_method',
                                         'fare_class_category'])
    
    results_clean = results_clean.sort_values(by=['transit_timestamp'])

    # ------------------------------------------------ #
    # GROUP & SUM DATATYPES
    # ------------------------------------------------ #
    
    # cast to float (starts as object)
    results_clean['ridership'] = results_clean['ridership'].astype('float')
    results_clean['transfers'] = results_clean['transfers'].astype('float')
    results_grouped = results_clean.groupby(by=["transit_timestamp", "station_complex_id", "station_complex", "borough"]).sum().reset_index()
    
    
    # ------------------------------------------------ #
    # SPLIT OUT DATETIME 
    # ------------------------------------------------ #
    
    results_grouped['transit_timestamp'] = pd.to_datetime(results_grouped['transit_timestamp'])
    
    results_grouped['year'] = pd.DatetimeIndex(results_grouped['transit_timestamp']).year
    results_grouped['month'] = pd.DatetimeIndex(results_grouped['transit_timestamp']).month
    results_grouped['day'] = pd.DatetimeIndex(results_grouped['transit_timestamp']).day
    results_grouped['hour'] = pd.DatetimeIndex(results_grouped['transit_timestamp']).hour
    results_grouped['dayofweek'] = pd.DatetimeIndex(results_grouped['transit_timestamp']).dayofweek
    
    results_grouped = results_grouped.reindex(columns=['transit_timestamp', 
                                                       'year',
                                                       'month',
                                                       'day',
                                                       'dayofweek',
                                                       'hour',
                                                       'ridership',
                                                       'transfers',
                                                       'station_complex_id',
                                                       'station_complex',
                                                       'borough'])
    
    
    # ------------------------------------------------ #
    # WRITE TO CSV
    # ------------------------------------------------ #
    
    results_grouped.to_csv(save_string_cleaned, index = False)

0131
0228
0331
0430
0531
0630
0731
0831
0930
1031
1130
1231


In [9]:
# ---------------------------------------------------------------------------------- #
# Concatenate 12 months of cleaned data of 1 station into a single csv for the year  
# --------------------------------------------------------------------------------- #

year_df = pd.DataFrame()

for m in months:
    file_string = './data/2023/cleaned/' +station_id+ '/' +station_name +'_' + m + '_cleaned.csv'
    df = pd.read_csv(file_string)
    year_df = pd.concat([year_df, df])

# ------------------------------------------------ #
# WRITE FULL YEAR TO CSV
# ------------------------------------------------ #

save_string = './data/2023/cleaned/' +station_id+ '/' +station_name +'_cleaned.csv'

year_df.to_csv(save_string, index = False)